In [227]:
import pandas as pd
from reco_systems.CF_knn import *
from sklearn.metrics.pairwise import manhattan_distances
from scipy.sparse import csr_array

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [288]:
folder = "database_cleaned"
avis_clean = pd.read_csv(f"database_cleaned/avis_clean.csv", index_col=0)
jeux_clean = pd.read_csv(f"database_cleaned/jeux_clean.csv", index_col=0)
#users = pd.read_csv(f"trictrac_database/users.csv", names=["Username", "User id"])
#users.shape

In [289]:
rev_filter, table_assoc_games= filter_games(avis_clean, min_reviews=10) 
#table_assoc.values, np.unique(rev_filter["User id"])
np.unique(rev_filter["User id"])
rev_filter[(rev_filter["Username"] == "limp") & (~rev_filter["Rating"].isna())].shape

(795, 8)

In [290]:
matrix_ratings, means = get_matrix_user_game(rev_filter)
matrix_ratings

/Users/bsh2022/Study/L3/Projet_recherche/Recommendation_system/reco_systems/CF_knn.py:37: RuntimeWarning: Mean of empty slice
  np.nan_to_num(matrix_ratings, copy=False, nan=0.0)  # replace nan values with 0.0


<Compressed Sparse Row sparse array of dtype 'float64'
	with 112559 stored elements and shape (11201, 2887)>

In [291]:
similarity_matrix = calc_similarity_matrix(matrix_ratings, dist_type="cos")
similarity_matrix.shape

(11201, 11201)

In [292]:
similar_users = get_KNN(similarity_matrix, k=30, user_ind=3000)
#similarity_matrix[83][np.argpartition(similarity_matrix[83], kth=min(10, similarity_matrix.shape[1]))[:10]]
similar_users

[0.84381735 0.83725311 0.84582188 ... 1.         1.01208626 1.        ]


array([ 1815,  6794,  7071,  4026,  1930,  6150,  7727,  1975, 10396,
        1939,   812,  6811, 10320,  3443,  2538,  4998,  4711,  2991,
        1938,  6626,  5019,  5710,  4618,  1229,  1906,  8018,  2608,
        2890,  1050,  2290])

In [293]:
for user in similar_users:
    print(user, np.count_nonzero(matrix_ratings.toarray()[user]))

1815 233
6794 6
7071 4
4026 3
1930 23
6150 6
7727 4
1975 11
10396 3
1939 215
812 49
6811 2
10320 2
3443 7
2538 117
4998 3
4711 4
2991 4
1938 172
6626 2
5019 13
5710 8
4618 7
1229 6
1906 285
8018 13
2608 5
2890 92
1050 53
2290 30


In [297]:
pred1=weight_avg_distance(similarity_matrix, similar_users, matrix_ratings, 3000, means)
pred2 = weight_avg_nb_reviews(rev_filter, similar_users, matrix_ratings, 3000, means)
ratings = predict_ratings(np.vstack((pred1, pred2)), None)

In [313]:
games_ind = get_games_ind(ratings, n=40)
games_ind

array([1461, 2271, 2790, 2775,  153, 1248, 1343, 2756, 2690,   95, 2658,
       1837, 2451,   88, 2178, 2380, 2058, 1297, 2456, 1117,  650, 2420,
       1018, 2834,  648, 1657,  504, 2563, 2651,  996, 1468, 1987, 1552,
        300, 1831, 1333, 2625,  445, 2649,  444])

In [314]:
df_pred_games = get_games_df(jeux_clean, table_assoc_games, games_ind)

In [315]:
avis_clean[["User id", "Game id", "Game name UI", "Rating"]][(avis_clean["User id"] == 3000)].merge(df_pred_games["Game id"], on="Game id")

,User id,Game id,Game name UI,Rating
0,3000,3577,Glastonbury,10.0
1,3000,3959,Herr der Ringe: Die Entscheidung,4.0
2,3000,10343,Ys,10.0
3,3000,373,Alhambra : 2 - Les Portes de la Ville,10.0
4,3000,4723,L'Escalier Hanté,8.0
